In [1]:
# imports
import pandas as pd
import gmaps
import requests

# api key
from config import g_key

In [2]:
# store csv into a DF
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Koumac,NC,2021-04-06 17:34:19,-20.5667,164.2833,73.35,77,100,5.73
1,1,Rikitea,PF,2021-04-06 17:32:34,-23.1203,-134.9692,76.84,73,67,10.69
2,2,Yellowknife,CA,2021-04-06 17:30:04,62.4560,-114.3525,18.00,79,90,18.41
3,3,Kem,RU,2021-04-06 17:39:34,64.9555,34.5793,31.87,73,99,14.72
4,4,Busselton,AU,2021-04-06 17:36:14,-33.6500,115.3333,60.01,96,76,1.01


In [3]:
# configure gmaps to use my Google API key
gmaps.configure(api_key = g_key)

In [4]:
# Heatmap of the temperature. <<< 

# get lat and lngs pair
locations = city_data_df[["Lat", "Lng"]]
# get the max temp
max_temp = city_data_df["Max Temp"]
temps = [max(temp, 0) for temp in max_temp]
#for temp in max_temp:
 #   temps.append(max(temp, 0))

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap var
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating = False, max_intensity=300, point_radius=4)

# add the heatmap layer
fig.add_layer(heat_layer)

# call the fig to plot the data
fig



Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap for % Humidity <<< 

locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap for % Cloudiness <<<

locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating = False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap for % Wind Speed <<<

locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating = False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# get inpust from max and min temp values
min_temp = float(input("Set min Temperature: "))
max_temp = float(input("Set max Temperature: "))

Set min Temperature: 65
Set max Temperature: 70


In [33]:
# filter the cities df to match the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df = preferred_cities_df.dropna()
#preferred_cities_df.count()

In [34]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [37]:
# Create DF called hotel_df to store hotel names along with 
# city, country, max temp, and coordinates.

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.count()

City          43
Country       43
Max Temp      43
Lat           43
Lng           43
Hotel Name    43
dtype: int64

In [38]:
# Setting the parameters for each hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# iterate throuugh the DF
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
    
Print("Done!")

Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Santa Cruz De Tenerife,ES,68.00,28.4682,-16.2546,Hotel Silken Atlántida Santa Cruz
13,Chokwe,MZ,68.14,-24.5333,32.9833,Complexo Sonho Real
24,Ponta Do Sol,PT,66.20,32.6667,-17.1000,Hotel do Campo
29,Khangah Dogran,PK,65.50,31.8317,73.6231,Dera Sabha Khokhar
50,Marsh Harbour,BS,69.42,26.5412,-77.0636,Abaco Beach Resort


In [44]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [45]:
# Heatmap for Temp Vaca Spots <<<

locations = hotel_df[["Lat", "Lng"]]
wind = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating = False, max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# call the fig to plot data
fig

Figure(layout=FigureLayout(height='420px'))